requiremenrs.txt

langchain
langchain-openai
langchainhub # langchain python라이브러리로 프롬프트, 에이전트, 체인 관련 패키지 모음
langserve[all]

faiss-cpu  # Facebook에서 개발 및 배포한 밀집 벡터의 유사도 측정, 클러스터링에 효율적인 라이브러리
tavily-python # 언어 모델에 중립적인 디자인으로, 모든 LLM과 통합이 가능하도록 설계된 검색 API
beautifulsoup4  #파이썬에서 사용할 수 있는 웹데이터 크롤링 라이브러리
wikipedia

fastapi #  Python의 API를 빌드하기 위한 웹 프레임워크
uvicorn # ASGI(Asynchronous Server Gateway Interface) 서버
urllib3 # 파이썬에서 HTTP 요청을 보내고 받는 데 사용되는 강력하고 유연한 라이브러리

python-dotenv
pypdf

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install python-dotenv
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu
!pip install wikipedia
!pip install openai
!pip install gradio

Tavily Search 를 사용하기 위해서는 API KEY를 발급 받아 등록해야 함.

[Tavily Search API 발급받기](https://app.tavily.com/sign-in)

발급 받은 API KEY 를 다음과 같이 환경변수에 등록

In [ ]:
import os

# TAVILY API KEY를 기입합니다.
os.environ["TAVILY_API_KEY"] = "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"

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "AGENT TUTORIAL"

In [ ]:
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
#google drive load
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import gradio as gr
import os
import openai
import unicodedata
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# ─── 0) OpenAI API 키 설정 ─────────────────────────────────────────────────
openai.api_key = os.getenv("OPENAI_API_KEY")

# ─── 1) JSON 로드 + 메타데이터 추출 함수 ─────────────────────────────────────────
def load_documents_with_metadata(folder_path):
    documents = []
    for raw_filename in os.listdir(folder_path):
        filename = unicodedata.normalize("NFC", raw_filename)
        file_path = os.path.join(folder_path, raw_filename)

        if not os.path.isfile(file_path):
            continue
        if not filename.endswith(".json"):
            continue

        try:
            parts = filename.replace(".json", "").split("_")
            emotion = parts[1] if len(parts) > 1 else "unknown"
            relation = parts[2] if len(parts) > 2 else "unknown"

            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
                utterances = data.get("utterances", [])
                full_text = "\n".join([utt.get("text","") for utt in utterances])
                if full_text.strip() == "":
                    continue

                doc = Document(
                    page_content=full_text,
                    metadata={"filename": filename, "emotion": emotion, "relation": relation}
                )
                documents.append(doc)
        except Exception as e:
            print(f"❌ 오류 발생 ({filename}): {e}")

    return documents

# ─── 2) 문서 분할 함수 ─────────────────────────────────────────────────────
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    return splitter.split_documents(documents)

# ─── 3) FAISS 인덱스 생성 혹은 로드 함수 ───────────────────────────────────
def create_or_load_faiss(index_dir, split_docs):
    embeddings = OpenAIEmbeddings()
    if os.path.isdir(index_dir) and os.path.exists(os.path.join(index_dir, "index.faiss")):
        faiss_db = FAISS.load_local(index_dir, embeddings, allow_dangerous_deserialization=True)
        print("✅ 기존 FAISS 인덱스를 로드했습니다.")
    else:
        faiss_db = FAISS.from_documents(split_docs, embeddings)
        os.makedirs(index_dir, exist_ok=True)
        faiss_db.save_local(index_dir)
        print("✅ 새로운 FAISS 인덱스를 생성하고 저장했습니다.")
    return faiss_db

# ─── 4) 필터 + 유사도 검색 함수 ────────────────────────────────────────────────
def filtered_similarity_search(vectorstore, query, emotion=None, relation=None, k=3):
    all_docs = vectorstore.docstore._dict.values()
    filtered_docs = [
        doc for doc in all_docs
        if (emotion is None or doc.metadata.get("emotion") == emotion)
        and (relation is None or relation in doc.metadata.get("relation"))
    ]

    if not filtered_docs:
        return []

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    query_chunks = splitter.split_text(query)

    search_results = []
    for chunk in query_chunks:
        search_results.extend(vectorstore.similarity_search(chunk, k=k))
    return search_results

# ─── 5) 후보 중 최고 문서 선택 함수 ─────────────────────────────────────────────
def choose_best_doc_with_gpt(query, docs, model="gpt-4o-mini"):
    prompt_parts = [
        "당신은 대화 응답 후보를 평가하는 전문가입니다.\n",
        f"사용자 질문: \"{query}\"\n",
        "다음은 검색된 응답 후보들입니다.\n"
    ]

    for idx, doc in enumerate(docs, start=1):
        snippet = doc.page_content.strip().replace("\n", " ")
        if len(snippet) > 300:
            snippet = snippet[:300] + "..."
        prompt_parts.append(
            f"[{idx}]\n"
            f"Filename: {doc.metadata.get('filename')}\n"
            f"Emotion: {doc.metadata.get('emotion')}, Relation: {doc.metadata.get('relation')}\n"
            f"Content: \"{snippet}\"\n"
        )

    prompt_parts.append(
        "\n위 후보들 중에서, 사용자 질문에 가장 적절한 응답을 하나 선택하고, 그 이유를 간단히 설명해주세요.\n"
        "반드시 다음 형식으로 응답해 주세요:\n"
        "선택: [번호]\n"
        "이유: [간단한 설명]\n"
    )

    full_prompt = "\n".join(prompt_parts)

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 뛰어난 대화 평가자입니다."},
            {"role": "user", "content": full_prompt}
        ],
        max_tokens=300,
        temperature=0.0
    )

    gpt_reply = response.choices[0].message.content.strip()
    selected_idx = None
    for line in gpt_reply.splitlines():
        if line.strip().startswith("선택"):
            import re
            m = re.search(r"\[(\d+)\]", line)
            if m:
                selected_idx = int(m.group(1))
                break

    if selected_idx is None or selected_idx < 1 or selected_idx > len(docs):
        selected_idx = 1

    best_doc = docs[selected_idx - 1]
    return best_doc, gpt_reply

# ─── 6) 최종 답변 간결하게 생성 함수 ─────────────────────────────────────────────
def generate_final_answer(query, best_doc, model="gpt-4o-mini"):
    prompt = (
        "다음은 사용자의 질문과, 선택된 최적 응답 후보입니다.\n\n"
        f"사용자 질문: \"{query}\"\n"
        "선택된 후보 응답 내용(원문):\n"
        f"\"\"\"\n{best_doc.page_content}\n\"\"\"\n\n"
        "위 원문에서, 불필요한 반복/인사말/개인정보 등은 모두 제거하고, "
        "사용자가 이해하기 쉽도록 핵심만 남겨 간결하게 재작성해주세요.\n"
        "문체는 친절하고 공감 가득한 톤을 유지해 주시고, "
        "최종 답변만 출력해 주세요."
    )

    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "당신은 친절하고 공감능력이 뛰어난 상담사입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )

    final_answer = response.choices[0].message.content.strip()
    return final_answer

# ─── 7) Gradio 응용: 채팅 인터페이스 구축 ───────────────────────────────────────
index_dir = "/content/drive/MyDrive/2025_Bigdata_nlp_class/faiss_index"
folder_path = "/content/drive/MyDrive/2025_Bigdata_nlp_class/aihub_dataset/Training/02_label_data"

# 문서 로드 및 FAISS 초기화
documents = load_documents_with_metadata(folder_path)
split_docs = split_documents(documents)
faiss_db = create_or_load_faiss(index_dir, split_docs)

def chat_response(query, emotion, relation):
    candidates = filtered_similarity_search(faiss_db, query, emotion, relation)
    if not candidates:
        return "조건에 맞는 문서가 없습니다."

    best_doc, _ = choose_best_doc_with_gpt(query, candidates, model="gpt-4o-mini")
    final_answer = generate_final_answer(query, best_doc, model="gpt-4o-mini")
    return final_answer

with gr.Blocks() as demo:
    gr.Markdown("## 감정/관계 기반 Empathy QA 시스템")
    with gr.Row():
        txt_query = gr.Textbox(label="질문", placeholder="질문을 입력하세요...", lines=2)
    with gr.Row():
        txt_emotion = gr.Textbox(label="Emotion (예: 기쁨, 당황, 분노)", placeholder="ex) 기쁨")
        txt_relation = gr.Textbox(label="Relation (예: 부모자녀, 부부, 연인)", placeholder="ex) 부모자녀")
    btn_submit = gr.Button("전송")
    output = gr.Textbox(label="답변", lines=5)

    btn_submit.click(chat_response, inputs=[txt_query, txt_emotion, txt_relation], outputs=output)

demo.launch()
